In [48]:
from keras.initializers import glorot_uniform
from keras.layers import Add, BatchNormalization, Concatenate, Conv2D, Input, ReLU
from keras.models import Model
from keras.regularizers import l2

from GuidedFilter import guided_filter

In [6]:
num_feature = 16             # number of feature maps
num_channels = 3             # number of input's channels 
patch_size = 64              # patch size 
KernelSize = 3               # kernel size 

In [6]:
learning_rate = 0.1          # learning rate
iterations = int(6e4)        # iterations
batch_size = 20              # batch size
save_model_path = "./model/" # saved model's path
model_name = 'model-epoch'   # saved model's name

In [53]:
def pre_process(images, base):
    base = guided_filter(images, images, 15, 1, nhwc=True) # using guided filter for obtaining base layer
    detail = images - base   # detail layer
    return detail

In [54]:
def DetailsNet(images, is_training):
    initializer = glorot_uniform()
    regularizer = l2(1e-10)
    
    conv2d = lambda layer_id: Conv2D(filters=num_feature, kernel_size=KernelSize, padding='same', 
                                     kernel_initializer=initializer, kernel_regularizer=regularizer,
                                     name='layer_%d/conv_%d' % (layer_id, layer_id))
    bn = lambda layer_id: BatchNormalization(name='layer_%d/bn_%d' % (layer_id, layer_id))
    relu = lambda layer_id: ReLU(name='layer_%d/relu_%d' % (layer_id, layer_id))
    
    inp = Input((images, images, num_channels))
    
    
    base = guided_filter(inp, inp, 15, 1, nhwc=True) # using guided filter for obtaining base layer
    detail = images - base   # detail layer
    
    output = conv2d(1)(detail)
    output = bn(1)(output, training=is_training)
    output_shortcut = relu(1)(output)
    
    #  layers 2 to 25
    for i in range(12):
        layer_id = i * 2 + 2
        output = conv2d(layer_id)(output_shortcut)
        output = bn(layer_id)(output, training=is_training)
        output = relu(layer_id)(output)

        layer_id = i * 2 + 3
        output = conv2d(layer_id)(output)
        output = bn(layer_id)(output, training=is_training)
        output = relu(layer_id)(output)

        output_shortcut = Add(name='add_%d'%i)([output_shortcut, output])

    # layer 26
    output = Conv2D(filters=num_channels, kernel_size=KernelSize, padding='same', 
                    kernel_initializer=initializer, kernel_regularizer=regularizer,
                    name='layer_26/conv_26')(output_shortcut)
    neg_residual = bn(26)(output, training=is_training)
    
    final_out = Add(name='add_final')([inp, neg_residual])
    
    return Model(inputs=[inp], outputs=[final_out])
    

In [55]:
model = inference(None, is_training=False)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
layer_1/conv_1 (Conv2D)         (None, 256, 256, 16) 448         input_14[0][0]                   
__________________________________________________________________________________________________
layer_1/bn_1 (BatchNormalizatio (None, 256, 256, 16) 64          layer_1/conv_1[0][0]             
__________________________________________________________________________________________________
layer_1/relu_1 (ReLU)           (None, 256, 256, 16) 0           layer_1/bn_1[0][0]               
__________________________________________________________________________________________________
layer_2/co

In [47]:
[1,2,3][-3:-2]

[1]

In [ ]:
def inference(images, is_training):
    regularizer = tf.contrib.layers.l2_regularizer(scale = 1e-10)
    initializer = tf.contrib.layers.xavier_initializer()


    base = guided_filter(images, images, 15, 1, nhwc=True) # using guided filter for obtaining base layer
    detail = images - base   # detail layer

   #  layer 1
    with tf.variable_scope('layer_1'):	
         output = tf.layers.conv2d(detail, num_feature, KernelSize, padding = 'same', kernel_initializer = initializer, 
                                   kernel_regularizer = regularizer, name='conv_1')
         output = tf.layers.batch_normalization(output, training=is_training, name='bn_1')
         output_shortcut = tf.nn.relu(output, name='relu_1')
  
   #  layers 2 to 25
    for i in range(12):
        with tf.variable_scope('layer_%d'%(i*2+2)):	
             output = tf.layers.conv2d(output_shortcut, num_feature, KernelSize, padding='same', kernel_initializer = initializer, 
                                       kernel_regularizer = regularizer, name=('conv_%d'%(i*2+2)))
             output = tf.layers.batch_normalization(output, training=is_training, name=('bn_%d'%(i*2+2)))	
             output = tf.nn.relu(output, name=('relu_%d'%(i*2+2)))

        with tf.variable_scope('layer_%d'%(i*2+3)): 
             output = tf.layers.conv2d(output, num_feature, KernelSize, padding='same', kernel_initializer = initializer,
                                       kernel_regularizer = regularizer, name=('conv_%d'%(i*2+3)))
             output = tf.layers.batch_normalization(output, training=is_training, name=('bn_%d'%(i*2+3)))
             output = tf.nn.relu(output, name=('relu_%d'%(i*2+3)))

        output_shortcut = tf.add(output_shortcut, output)	# shortcut

   # layer 26
    with tf.variable_scope('layer_26'):
         output = tf.layers.conv2d(output_shortcut, num_channels, KernelSize, padding='same',   kernel_initializer = initializer, 
                                   kernel_regularizer = regularizer, name='conv_26')
         neg_residual = tf.layers.batch_normalization(output, training=is_training, name='bn_26')
		 
    final_out = tf.add(images, neg_residual)
	
    return final_out